In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os

# --- 1. IMPORT DANYCH WEJŚCIOWYCH ---
print("Log: Inicjalizacja procesu przygotowania danych (Data Preparation)...")

try:
    df = pd.read_csv('../data/lending_club_cleaned.csv')
    print(f"Log: Poprawnie wczytano zbiór danych. Wymiary: {df.shape[0]} wierszy, {df.shape[1]} kolumn.")
except FileNotFoundError:
    print("BŁĄD KRYTYCZNY: Nie odnaleziono pliku 'lending_club_cleaned.csv' w katalogu data.")
    raise

# --- 2. SELEKCJA ZMIENNYCH (FEATURE SELECTION) ---

# Usuwanie zmiennych, które nie mogą brać udziału w modelowaniu:
# - Zmienne wynikowe (target, loan_status)
# - Zmienne powodujące wyciek danych (Data Leakage): grade, int_rate, installment
# - Zmienne techniczne/czasowe wymagające zaawansowanej inżynierii (upraszczamy model)

features_to_drop = [
    'loan_status', 'target',          # Zmienna objaśniana (y)
    'grade', 'sub_grade', 'int_rate', # Ocena ryzyka nadana przez bank (niedostępna w momencie wnioskowania)
    'installment',                    # Pochodna oprocentowania i kwoty
    'issue_d', 'earliest_cr_line',    # Zmienne czasowe
    'fico_range_low', 'fico_range_high' # Zastąpione przez uśredniony 'fico_score'
]

# X - Macierz zmiennych objaśniających
X = df.drop(columns=features_to_drop)

# y - Wektor zmiennej objaśnianej (Target)
y = df['target']

print(f"\nLog: Wyselekcjonowano zmienne wejściowe. Liczba cech pierwotnych: {X.shape[1]}")
# print(list(X.columns)) # Opcjonalnie: wyświetlenie listy kolumn

# --- 3. KODOWANIE ZMIENNYCH KATEGORIALNYCH (ONE-HOT ENCODING) ---
# Algorytmy ML wymagają danych numerycznych. Zamiana zmiennych tekstowych na binarne.

X = pd.get_dummies(X, drop_first=True)
print(f"Log: Liczba zmiennych po zakodowaniu (One-Hot Encoding): {X.shape[1]}")

# --- 4. PODZIAŁ NA ZBIÓR TRENINGOWY I TESTOWY (TRAIN-TEST SPLIT) ---
# Podział danych w proporcji 80:20 z zachowaniem warstwowania (stratify) względem zmiennej celu.
# Zapewnia to identyczny odsetek kredytów niespłaconych w obu zbiorach.

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n--- PODSUMOWANIE PODZIAŁU DANYCH ---")
print(f"Zbiór Treningowy (Training Set): {X_train.shape[0]:,} obserwacji".replace(',', ' '))
print(f"Zbiór Testowy (Test Set):        {X_test.shape[0]:,} obserwacji".replace(',', ' '))

# --- 5. EKSPORT PRZETWORZONYCH DANYCH ---
# Zapis przygotowanych macierzy do plików CSV w celu wykorzystania w modelach.
print("\nLog: Trwa zapisywanie plików wynikowych do katalogu data...")

X_train.to_csv('../data/X_train.csv', index=False)
X_test.to_csv('../data/X_test.csv', index=False)
y_train.to_csv('../data/y_train.csv', index=False)
y_test.to_csv('../data/y_test.csv', index=False)

# Eksport pełnych danych testowych (z oprocentowaniem) dla analizy biznesowej (Business Case)
df_test_full = df.loc[X_test.index]
df_test_full.to_csv('../data/df_test_full_check.csv', index=False)

print("Proces zakończony pomyślnie. Dane są gotowe do etapu modelowania.")

Log: Inicjalizacja procesu przygotowania danych (Data Preparation)...
Log: Poprawnie wczytano zbiór danych. Wymiary: 1345350 wierszy, 23 kolumn.

Log: Wyselekcjonowano zmienne wejściowe. Liczba cech pierwotnych: 13
Log: Liczba zmiennych po zakodowaniu (One-Hot Encoding): 39

--- PODSUMOWANIE PODZIAŁU DANYCH ---
Zbiór Treningowy (Training Set): 1 076 280 obserwacji
Zbiór Testowy (Test Set):        269 070 obserwacji

Log: Trwa zapisywanie plików wynikowych do katalogu data...
Proces zakończony pomyślnie. Dane są gotowe do etapu modelowania.


W ramach przygotowania danych do modelowania wykonano następujące kroki:

1.  **Eliminacja wycieku danych:**
    Usunięto zmienne: `grade`, `sub_grade`, `int_rate`, `installment`. Są to oceny ryzyka nadawane przez bank *po* złożeniu wniosku. Ich uwzględnienie sprawiłoby, że model "oszukiwałby", znając odpowiedź z góry.

2.  **Weryfikacja hipotezy o FICO:**
    Analiza korelacji potwierdziła, że wynik FICO jest niezależny od dochodu (`corr = 0.07`) oraz wskaźnika zadłużenia DTI. Potwierdza to sens budowy modelu (dodanie danych dochodowych wniesie nową informację).

3.  **Transformacja zmiennych:**
    Zmienne kategoryczne (np. cel kredytu, typ własności mieszkania) zostały zamienione na postać numeryczną metodą One-Hot Encoding. Wymiarowość zbioru wzrosła z 13 do 39 cech.

4.  **Podział zbioru:**
    Dane podzielono w proporcji 80:20 z zastosowaniem stratyfikacji (`stratify=y`).
    * **Zbiór Treningowy:** 1.07 mln obserwacji (do nauki algorytmów).
    * **Zbiór Testowy:** 269 tys. obserwacji (do finalnej walidacji).

**notatka do pracy**

Proces transformacji surowego zbioru danych do postaci akceptowalnej przez algorytmy uczenia maszynowego zakończono utworzeniem macierzy wejściowej składającej się z 39 zmiennych objaśniających (features).

Kluczowym elementem metodologicznym było usunięcie ze zbioru treningowego zmiennych będących pochodną decyzji kredytowej (takich jak ocena grade czy oprocentowanie int_rate). Zabieg ten, mający na celu uniknięcie zjawiska wycieku informacji (Data Leakage), gwarantuje, że budowane modele będą symulować rzeczywisty proces decyzyjny, opierając się wyłącznie na informacjach dostępnych w momencie składania wniosku kredytowego.

Potwierdzono również empirycznie (poprzez analizę korelacji r-Pearsona), że tradycyjna ocena punktowa FICO nie zawiera w sobie informacji o dochodach wnioskodawcy, co stanowi uzasadnienie dla włączenia zmiennych finansowych do modelu eksperymentalnego (Challenger).

Ostateczny zbiór danych został podzielony na podzbiór treningowy (80%) i testowy (20%) metodą losowania warstwowego (stratified sampling), co zapewniło zachowanie identycznej struktury ryzyka (proporcji kredytów niespłaconych) w obu próbach badawczych.